In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

e:\apps\anaconda\envs\dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_step = 2 # number of steps, n-1 in paper
n_hidden = 2 # number of hidden size, h in paper
m = 2 # embedding size, m in paper

sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)  # number of Vocabulary

In [3]:
def make_batch():
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        words = sen.split()
        # 语料索引化
        input = [word_dict[n] for n in words[:-1]]
        target = word_dict[words[-1]]
        
        input_batch.append(input)
        target_batch.append(target)
    
    return input_batch,target_batch


In [4]:
class NNLM(nn.Module):
    def __init__(self):
        # 调用父类的构造函数
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, m)
        self.H = nn.Linear(n_step*m, n_hidden, bias=False)
        self.d = nn.Parameter(torch.ones(n_hidden))
        self.U = nn.Linear(n_hidden, n_class, bias=False)
        self.W = nn.Linear(n_step*m, n_class, bias=False)
        self.b = nn.Parameter(torch.ones(n_class))
    
    def forward(self, X):
        X = self.C(X) # X: [batch_size, n_step, n_class] => [batch_size, n_step, m]
        X = X.view(-1, n_step*m) # [batch_size, n_step, m] => [batch_size, n_step*m]
        tanh = torch.tanh(self.d + self.H(X)) # [batch_size, n_step*m] => [batch_size, n_hidden]
        output = self.b + self.W(X) + self.U(tanh) # [batch_size, n_hidden] => [batch_size, n_class]
        
        return output
        

In [7]:
model = NNLM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch()
input_batch = torch.LongTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

Traceback (most recent call last):
  File "e:\apps\anaconda\envs\dl\lib\site-packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_vars.py", line 478, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
  File "<string>", line 1, in <module>
NameError: name 'tensor' is not defined


In [11]:
# Training
for epoch in range(5000):
    optimizer.zero_grad()
    output = model(input_batch)

    # output : [batch_size, n_class], target_batch : [batch_size]
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()


Traceback (most recent call last):
  File "e:\apps\anaconda\envs\dl\lib\site-packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_vars.py", line 478, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
  File "<string>", line 1, in <module>
NameError: name 'tensor' is not defined


Epoch: 1000 cost = 0.000395
Epoch: 2000 cost = 0.000221
Epoch: 3000 cost = 0.000126
Epoch: 4000 cost = 0.000073
Epoch: 5000 cost = 0.000042


In [9]:
# Predict
predict = model(input_batch).data.max(1, keepdim=True)[1]


In [10]:
# Test
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']


在 PyTorch 中，`squeeze` 函数是一个非常有用的工具，用于去除张量中所有维度为 1 的维度。这在处理维度为 1 的批次大小或序列长度时尤其有用。让我们看看 `squeeze` 的常见用法和一些示例：

### 常见用法

1. **去除所有单维度**：
   - 当 `squeeze` 被应用于一个张量而没有指定维度时，它会去除张量中所有维度为 1 的维度。

   示例：
   ```python
   x = torch.zeros(2, 1, 2, 1, 2)  # 形状为 [2, 1, 2, 1, 2]
   y = x.squeeze()  # y 的形状将变为 [2, 2, 2]
   ```

2. **指定维度去除**：
   - 你也可以指定要去除的维度。如果指定的维度不是单维度，则张量不会改变。

   示例：
   ```python
   x = torch.zeros(2, 1, 2)  # 形状为 [2, 1, 2]
   y = x.squeeze(1)  # y 的形状将变为 [2, 2]
   z = x.squeeze(0)  # z 的形状仍然是 [2, 1, 2]，因为维度 0 不是单维度